# Dependences

In [21]:
include("../../src/struct_data.jl")

save_data

# Model

In [22]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 100000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 50
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_4_MiniTest",
        path_output = ""
    ) 
)

dump(model)

  0.000003 seconds (4 allocations: 144 bytes)
ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 100000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 100
    nₛₐᵥₑ: Int64 50
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"
  Output: OutputModel
    name_output: String "Test_4_MiniTest"
    path_output: String ""


In [23]:
# Par1 ,Par2 = Cubic(0.01,2.0,3.0), ContractilePar(0.06)
Par1 ,Par2 = Cubic(0.01,2.0,3.0), ContractilePar(0.06)
agg_size = 15

15

# Aggregate

In [24]:
# Run only one aggregate
agg = nothing
agg = Aggregate(
    [AggType(
        "HEK_1", 
        InteractionPar(Par1, Par2),
        Float32.(readdlm("../../data/init/Sphere/$(agg_size).0.xyz")[3:end,2:end]) |> cu
    )], 
    [AggLocation("HEK_1",[0 0 0]),],
    model
)

Aggregate(AggType[AggType("HEK_1", InteractionPar(Cubic{Float64}(0.01, 2.0, 3.0), ContractilePar(0.06)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})], AggIndex([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], ["HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"  …  "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"]), Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], AggGeometry(Float32[15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27  …  15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), AggSimulation(AggParameter(Cubic{CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}}(Float32[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 

In [25]:
# # Run fusion of two aggregates
# agg = nothing
# agg = FusionAggregate(
#     [AggType(
#         "HEK_1", 
#         InteractionPar(Par1, Par2),
#         Float32.(readdlm("../../data/init/Sphere/$(agg_size).0.xyz")[3:end,2:end]) |> cu
#     )], 
#     model
# )

# kNN

In [26]:
include("../../src/neighbor.jl")

dist = zeros(size(agg.Simulation.Neighbor.idx_red)) |> cu
threads=(100)
@cuda(
    threads=threads,
    blocks=cld.(size(agg.Position,1),threads),
    dist_kernel!(
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Neighbor.idx_cont,
        agg.Simulation.Neighbor.idx_sum,
        agg.Simulation.Neighbor.dist,
        agg.Position,
        agg.Simulation.Parameter.Force.rₘₐₓ
    )
)   

display(agg.Simulation.Neighbor.idx_red)

1000×2504 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 36  37  38  41  42  43  44  45  57  …  2461  2462  2463  2468  2469  2470
  6   7   8  49   1   2   3   3  58     2502  2503  2504  2498  2499  2500
  5   6   7  50  50   1   2  53  15     2503  2504  2455  2499  2500  2501
 42  43  44   5  51  51  52  54  10     2454  2455  2456  2461  2462  2463
 43  44  45  10   6  52  53   7  49     2453  2454  2500  2460  2461  2462
  2   3   2   9   4   7   8  14   4  …  2498  2499  2495  2503  2502  2503
 35   1  37  40  11   5   6  13  66     2500  2501  2496  2469  2504  2471
 37  36  39  42  10  12  13  44  50     2493  2494  2462  2467  2468  2469
 51  38  53  58  41  11  12  46  48     2494  2495  2464  2453  2470  2455
  0  52   0   0  43  42  43  62   0     2460  2461  2447     0  2454     0
  0   0   0   0  59  44  45   0   0  …  2462  2463     0     0     0     0
  0   0   0   0   0  60  61   0   0     2445  2446     0     0     0     0
  0   0   0   0   0   0   0   0   0        0    

# Sum Forces

In [27]:
include("../../src/forces/forces.jl")

threads=(64,3)
CUDA.@time @cuda(
    threads = threads,
    blocks = cld.(size(agg.Position,),threads),
    shmem=prod(threads.+2)*sizeof(Float32),
    sum_force!(
        agg.Position,
        agg.Simulation.Force.F,
        agg.Simulation.Force.Pol,
        agg.Simulation.Force.N_i,
        agg.Simulation.Neighbor.idx_sum,
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Parameter.Force,
        agg.Simulation.Parameter.Contractile.fₚ,
        atan(1),
        pi/4,
        model.Time.dt
    )
)

display(agg.Simulation.Force.F)

  0.669083 seconds (348.88 k CPU allocations: 20.681 MiB)


2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  0.043071     -0.0294104    -0.0293135
 -0.133309      0.115847      0.0350745
  7.13207f-12  -1.49224f-5    0.000364654
 -4.6549f-12    0.000133741   0.000364654
  0.0768459    -0.00681949    0.0922601
  0.0465833    -0.0182006    -0.0326916
 -0.025562     -0.112096      0.03419
 -0.0279431     0.047044     -0.0245264
  0.0107768     0.0558033    -0.0186341
  0.12248      -0.0818346     0.10394
 -0.0281172     0.116157      0.134827
  0.0473493     0.0493741     0.0988765
 -0.10265       0.0502115    -0.0360465
  ⋮                          
 -0.0868456    -0.113108     -0.110039
 -0.011029     -0.112105      0.0405655
 -0.00998787    0.154583     -0.0923076
  0.0210768    -0.0106786    -0.0554844
  0.0243207     0.105448     -0.0519487
 -0.0847751    -0.0423247    -0.0740933
  0.0270936     0.0533657     0.000297269
  0.11316       0.0254341    -0.0310149
 -0.157235      0.00842559   -0.0875467
  0.0764169    -0.124789     -0.105171


# MiniTest

In [28]:
include("../../src/run_event.jl")

run_test(agg, model, "mini Testing")
display(sum(isnan.(agg.Simulation.Force.F), dims=1))
display(sum(agg.Simulation.Force.F .> 50, dims=1))
display(agg.Simulation.Force.F)
display(agg.Position)

mini Testing... 100%|████████████████████████████████████| Time: 0:01:29


1×3 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 0  0  0

1×3 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 0  0  0

2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  0.124231    -0.0179999   -0.0415326
  0.00726705  -0.0606817   -0.0525248
 -0.0526137   -0.0736159    0.0584535
 -0.0439396    0.0378478   -0.0351567
  0.045477    -0.00447639   0.0994545
  0.00281543  -0.00540495  -0.0220775
 -0.120806     0.122272    -0.0154183
  0.0523234    0.0429672   -0.0127826
  0.00432141  -0.0564628   -0.0501972
  0.034486    -0.0440945   -0.0437287
  0.0459258   -0.0290485   -0.0566592
  0.00417353  -0.0726629    0.0692204
 -0.0185805   -0.0427456   -0.0395422
  ⋮                        
  0.0832886   -0.0393252   -0.0494005
 -0.0631794    0.0669523   -0.0446904
 -0.00632308   0.00689699   0.0213604
  0.0798737   -0.0151113    0.0249002
 -0.058244    -0.0396251   -0.0767344
 -0.0917219    0.0767286    0.0439196
  0.0848986    0.123316    -0.0738111
 -0.0639895   -0.147726    -0.0531775
 -0.00544937   0.0522947    0.0556464
 -0.00890012  -0.0460745    0.0268725
 -0.0436816   -0.0689693   -0.0497022
  0.00555

2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -0.294962   -3.73715   -12.7061
  1.55234    -0.771155  -12.7692
  2.43138    -2.25629   -12.4982
 -4.31124    -3.40049   -11.3831
  0.317371   -5.75777   -11.6569
  0.772631   -2.27017   -12.8688
  1.07247    -1.54744   -11.5062
  3.89481    -2.28572   -12.6552
 -3.52708    -4.77349   -11.4064
 -2.32792    -2.41845   -12.6553
 -0.804733   -2.19639   -12.5321
  0.0687602  -0.811087  -12.6471
  3.17612    -0.824996  -12.6821
  ⋮                     
  0.673809   -0.00523     8.64402
  2.13308     1.36743     7.15905
  3.77264     0.711998    8.47094
  5.09958     0.330191    8.32646
  0.0275699   5.27001     7.85974
 -2.88413    -0.330659    8.24535
 -0.164698    1.62179     8.39735
  2.28041     0.213101    8.38472
  5.25824     1.46011     7.20266
 -1.21118     2.84297     8.77559
 -0.70162     6.59997     7.72015
  0.745622    3.84975     7.52995